In [15]:
import pandas as pd

In [16]:
stablecoins = pd.read_json('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&category=stablecoins&order=market_cap_desc&per_page=100&page=1&sparkline=false&locale=en')
maxvolume   = pd.read_json('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=volume_desc&per_page=100&page=1&sparkline=false&locale=en')
market_cap  = pd.read_json('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false&locale=en')

In [17]:
stablecoins = list(stablecoins['id'])
stablecoins

['tether',
 'usd-coin',
 'dai',
 'binance-usd',
 'true-usd',
 'frax',
 'usdd',
 'paxos-standard',
 'pax-gold',
 'tether-gold',
 'first-digital-usd',
 'gemini-dollar',
 'liquity-usd',
 'tether-eurt',
 'eusd-27a558b0-8b5b-4225-a614-63539da936f4',
 'alchemix-usd',
 'stasis-eurs',
 'terrausd',
 'bilira',
 'magic-internet-money',
 'bob',
 'dola-usd',
 'xsgd',
 'flex-usd',
 'helio-protocol-hay',
 'nusd',
 'bean',
 'fei-usd',
 'token-dforce-usd',
 'mimatic',
 'r',
 'origin-dollar',
 'ageur',
 'tor',
 'zusd',
 'celo-euro',
 'gyen',
 'yusd-stablecoin',
 'usdx',
 'wemix-dollar',
 'husd',
 'vesta-stable',
 'celo-dollar',
 'straitsx-indonesia-rupiah',
 'djed',
 'vai',
 'seur',
 'musd',
 'cnh-tether',
 'usdp',
 'par-stablecoin',
 'equilibrium-eosdt',
 'spiceusd',
 'sperax-usd',
 'coin98-dollar',
 'usdk',
 'reserve',
 'zasset-zusd',
 'basis-cash',
 'statik',
 'one-cash',
 'dynamic-set-dollar',
 'usd-balance',
 'xdollar-stablecoin',
 'binance-peg-busd',
 'bdollar',
 'xdai',
 'monerium-eur-money',
 'k

In [18]:
maxvolume  = list(maxvolume [~maxvolume['id'].isin(stablecoins)]['id'])
market_cap = list(market_cap[~market_cap['id'].isin(stablecoins)]['id'])

In [19]:
# Selecciona las 8 monedas con mas vólumen
coins = maxvolume[:8]
i = 0

#Agrego las 7 intercaladas con mayor market_cap que no existan en coins
while len(coins)<15:
    if market_cap[i] not in coins:
        coins.insert(i+1, market_cap[i])
    i += 1


In [20]:
print(coins)
print(len(coins))

['aave-eth-v1', 'bitcoin', 'ethereum', 'binancecoin', 'aave-usdc-v1', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot', 'aave-link-v1', 'aave-tusd-v1', 'ripple', 'immutable-x']
15


In [21]:
coins.remove('aave-eth-v1')
coins.remove('aave-usdc-v1')
coins.remove('aave-link-v1')
coins.remove('aave-tusd-v1')

In [22]:
coins = coins [:10]
coins

['bitcoin',
 'ethereum',
 'binancecoin',
 'staked-ether',
 'cardano',
 'dogecoin',
 'solana',
 'tron',
 'polkadot',
 'ripple']

In [23]:
df = pd.DataFrame()

# 5 años
dias = 365 * 5 + 1

for moneda in coins:

    url = f'https://api.coingecko.com/api/v3/coins/{moneda}/market_chart?vs_currency=usd&days={dias}&interval=daily&precision=3'
    df_tmp = pd.read_json(url)
    df_tmp ['id_coin'] = moneda
    # Elimina el último registro del día incompleto
    df_tmp = df_tmp[:-1]


    if len(df) > 0:
        df = pd.concat([df, df_tmp], ignore_index=True)
    else:
        df = df_tmp



In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16093 entries, 0 to 16092
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   prices         16093 non-null  object
 1   market_caps    16093 non-null  object
 2   total_volumes  16093 non-null  object
 3   id_coin        16093 non-null  object
dtypes: object(4)
memory usage: 503.0+ KB


In [25]:
df['date'] = df.prices.apply(lambda x: pd.to_datetime(x[0], unit='ms'))
df['date'] = pd.to_datetime(df['date'])
df['prices'] = df.prices.apply(lambda x: x[1])
df['market_caps'] = df.market_caps.apply(lambda x: x[1])
df['total_volumes'] = df.total_volumes.apply(lambda x: x[1])

In [26]:
df.to_csv(f'data/cotizaciones.csv', index=False)

In [27]:

coins = df['id_coin'].unique()
rows = []
for c in coins:
    rows.append([c, \
                df[df['id_coin']==c]['date'].min(),\
                df[df['id_coin']==c]['prices'].max(),\
                df[df['id_coin']==c]['prices'].min() ])
    

    
df2 = pd.DataFrame(rows, columns=['id_coin', 'min_date', 'max_price', 'min_price'])



df2.to_csv('data/monedas.csv', index=False)